In [1]:
import pandas as pd
import numpy as np
import copy 

In [2]:
df = pd.read_csv("Tape_Deletion_Datasets")

In [3]:
datasets = df["dataset"]
location = df["rse_id"]
tier = df["DTier"]
size_on_tape = df["size_on_tape_[TB]"]

In [4]:
exceptions = pd.read_csv("exceptions.txt")

In [5]:
exception_list = exceptions["item"]

## Total size on tape

In [6]:
np.sum(size_on_tape)

226217.55768000003

In [7]:
keep = np.full( np.size(datasets), False )

## Breakdown of campaign to "human readable" format

In [8]:
categories = {

    "Skims":["USER"],
    "Commissioning + Cosmics":[
       "Commissioning08",     
       "Commissioning10",  
       "Commissioning11",  
       "Commissioning12",  
       "Commissioning2015",
       "Commissioning2017",
       "Commissioning2018",
       "Commissioning2021",
       "Commissioning2022",
    ],
        
    "Run 3":[
        "Run3Summer19DRPremix",       
        "Run3Summer19DR",  
        "Run3Winter20",
        "Run3Winter20DRPremixMiniAOD",
        "Run3Winter21DRMiniAOD",      
        "Run3Summer21",               
        "Run3Winter21",               
        "Run3Summer21GS",             
        "Run3Summer21PrePremix",      
        "Run3Summer21wmLHEGS",        
        "Run3Summer22",                  
    ],
    
    "Run 2 legacy MC":[
        "Summer19UL",
        "Summer20UL",
        "Spring21UL",
    ],
    
    "Run 2 legacy data":[
        "UL2016",
        "UL2017",
        "UL2018",
    ],
    
    "Run 2 pre-legacy data":[
        "23Sep2016",
        "18Apr2017",
        "07Aug17",
        "12Sep2017",
        "17Nov2017",
        "09May2018",
        "06Jun2018",
        "17Sept2018",
        "22Jan2019",
        "Run2018D-PromptReco"
    ],

    "Run 2 pre-legacy MC":[
        "RunIISpring15PrePremix",    
        "RunIIFall15DR76",           
        "RunIISummer16DR80",         
        "RunIISummer16DR80Premix",   
        "RunIIFall17GS",             
        "RunIIFall17DRPremix",       
        "RunIISummer17PrePremix",    
        "RunIIFall17FSPremix",       
        "RunIIFall17FSPrePremix",    
        "RunIIFall18wmLHEGS",        
        "RunIIFall18GS",             
        "RunIIAutumn18DRPremix",     
        "RunIIAutumn18DR",           
        "RunIIAutumn18RECOBParking", 
        "RunIIAutumn18FSPremix",        
        "RunIIWinter19PFCalib17wmLHEGS",
        "RunIISpring18DRPremix",        
        "RunIIFall17DRStdmix",          
        "RunIISummer17DRStdmix",        
        "RunIISpring18DR",              
        "RunIISpring15DR74",            
        "RunIISummer16FSPremix",        
        "RunIISummer15wmLHEGS",         
        "RunIIWinter19CosmicDR",        
        "RunIIFall17wmLHEGS",           
        "RunIIWinter19PFCalib16wmLHEGS",
        "RunIIFall17wmLHEGENOnly",      
        "RunIIWinter15GS",              
    ],
    
    "B-Parking":[
        "ParkingBPH"
    ],    
    
    "HIN":[
        "HIRun2010", 
        "HIRun2011", 
        "HIRun2013", 
        "HIRun2015", 
        "PARun2016", 
        "HIRun2018", 
        "HINPbPbAutumn18DR",
        "HiWinter13", 
        "pPb816Summer16DR",
        "HINPbPbWinter16DR",
        "HiFall13DR53X",
        "pAWinter13DR53X",
        "XeXeRun2017",
        "HIRun2013A",
        "HINppWinter16DR",
    ],

    
    "DPOA":[
        "LowPU2010",
        "Run2010",
        "Run2010",
        "Run2010",
        "Run2011",
        "Run2011",
        "Run2011",
        "Run2012",
        "Run2012",
        "Summer11",
        "Summer11Leg",
        "Fall11",
        "Summer11LegDR",
        "Summer12DR53X",
        "Summer11dr53X",
        "Summer12_DR53X",
        "Summer12",
        "Summer12",
        "Summer13dr53X",
        "Run2015C_25ns",
        "Run2015",
        "Run2015",
    ],
    
    "Upgrades":[
        "Phase2HLTTDRWinter20DIGI", 
        "Phase2HLTTDRSummer20ReRECOMiniAOD",
        "PhaseIITDRSpring17DR", 
        "PhaseIIFall16DR82",
        "PhaseIISummer17wmLHEGENOnly",
        "GEM2019Upg14DR",
        "PhaseISpring17DR", 
        "PhaseIFall16DR", 
    ],
    
    "ALCA":[
        "StreamExpressAlignment",
        "TkAlCosmicsInCollisions",
        "ALCARECO"
    ],

    "Unknown":[
        "CMSSW_1_3_2",
    ],
}



### List of all campaigns

In [9]:
campaign_list = []

for vals in categories.values():
    campaign_list = campaign_list + vals

### Map of campaigns to the category they belong to

In [10]:
cats_map = {}
for val in campaign_list: 
    for cat in categories:
        if val in categories[cat]:
            cats_map[val] = cat

### Map of datasets to the category they belong to

In [11]:
dataset_categories = []
for dataset in datasets:
    found = False
    for category in categories: 
        for campaign in categories[category]:
            if campaign in dataset and not found: 
                dataset_categories.append(category)
                found = True
    if not found:
        dataset_categories.append("Unknown")

In [12]:
df['category'] = dataset_categories

In [13]:
for category in categories:
    isum = np.sum( df[ df["category"] == category]["size_on_tape_[TB]"] )
    print( "%30s : %12.1f" % ( category, isum ) )

                         Skims :       4063.3
       Commissioning + Cosmics :       1476.0
                         Run 3 :       8562.9
               Run 2 legacy MC :      46427.0
             Run 2 legacy data :      15986.4
         Run 2 pre-legacy data :      13005.4
           Run 2 pre-legacy MC :      50326.7
                     B-Parking :       6369.3
                           HIN :      11001.6
                          DPOA :      12886.3
                      Upgrades :      12661.2
                          ALCA :       4739.9
                       Unknown :      38711.4


## Decide which datasets to delete and keep

#### Specific exceptions

In [14]:
retained1 = np.in1d(datasets, exception_list)

In [15]:
keep = np.where( retained1 == True, True, keep )

#### Global exceptions

In [16]:
global_exceptions = {}

for campaign in campaign_list:
    global_exceptions[campaign] = []

In [17]:
campaign_tiers_to_keep = [
        ["USER", "USER"],                         #
        ["Commissioning08", "*"],                 #    
        ["Commissioning10", "*"],                 #
        ["Commissioning11", "*"],                 #
        ["Commissioning12", "RECO"],              #
        ["Commissioning2015", "*"],               # 
        ["Commissioning2017", "AOD"],             #
        ["Commissioning2018", "RAW-RECO"],        #
        ["Commissioning2021", "*"],                # 
        ["Commissioning2022", "*"],                # 
        ["Run3Summer19DRPremix", "*"],       #
        ["Run3Summer19DR", "*"],             #
        ["Run3Winter20", "*"],
        ["Run3Winter20DRPremixMiniAOD", "AODSIM"],#
        ["Run3Winter21DRMiniAOD", "AODSIM"],       # 
        ["Run3Summer21", "*"],                    #
        ["Run3Winter21", "*"],                    #
        ["Run3Summer21GS", "*"],                  #
        ["Run3Summer21PrePremix", "*"],           #
        ["Run3Summer21wmLHEGS", "GEN-SIM"],       #
        ["Run3Summer22", "*"],                    #    
        ["Summer20UL", "*"],                       # 
        ["Spring21UL", "*"],                       # 
        ["UL2016", "*"],                           # 
        ["UL2017", "*"],                           # 
        ["UL2018", "*"],                           #
        ["23Sep2016", "AOD"], 
        ["18Apr2017", "AOD"], 
        ["07Aug17", "AOD"], 
        ["12Sep2017", "AOD"], 
        ["17Nov2017", "AOD"], 
        ["09May2018", "AOD"], 
        ["06Jun2018", "AOD"], 
        ["17Sept2018", "AOD"], 
        ["22Jan2019", "AOD"], 
        ["Run2018D-PromptReco", "AOD"], 
        ["Run2018D-PromptReco", "AOD"],    
        ["RunIISpring15PrePremix", "*"],     #
        ["RunIIFall15DR76", "AODSIM"],            #
        ["RunIISummer16DR80", "AODSIM"],          #     
        ["RunIISummer16DR80Premix", "*"],     #
        ["RunIIFall17GS", "*"],                   #
        ["RunIIFall17DRPremix", "*"],         # 
        ["RunIISummer17PrePremix", "*"],      #
        ["RunIIFall17FSPremix", "*"],             #
        ["RunIIFall17FSPrePremix", "*"],          #
        ["RunIIFall18wmLHEGS", "*"],              #
        ["RunIIFall18GS", "*"],                   #
        ["RunIIAutumn18DRPremix", "*"],      # 
        ["RunIIAutumn18DR", "AODSIM"],            #
        ["RunIIAutumn18RECOBParking", "AODSIM"],  #
        ["RunIIAutumn18FSPremix", "*"],      #
        ["RunIIWinter19PFCalib17wmLHEGS", "LHE"], #
        ["RunIISpring18DRPremix", "*"],      #
        ["RunIIFall17DRStdmix", "*"],        #
        ["RunIISummer17DRStdmix", "*"],      #
        ["RunIISpring18DR", "AODSIM"],            #
        ["RunIISpring15DR74", "AODSIM"],          #
        ["RunIISummer16FSPremix", "*"],      #
        ["RunIISummer15wmLHEGS", "LHE"],          #
        ["RunIIWinter19CosmicDR", "*"],           #
        ["RunIIFall17wmLHEGS", "LHE"],            #    
        ["RunIIWinter19PFCalib16wmLHEGS", "*"],   #
        ["RunIIFall17wmLHEGENOnly", "LHE"],       #
        ["RunIIWinter15GS", "GEN-SIM"],           #
        ["ParkingBPH", "AOD"],                     # 
        ["HIRun2010", "RECO"],                    # 
        ["HIRun2011", "AOD"],                     #
        ["HIRun2013", "AOD"],                     #
        ["HIRun2015", "AOD"],                     #
        ["PARun2016", "AOD"],                     #
        ["HIRun2018", "AOD"],                     #
        ["HINPbPbAutumn18DR", "AODSIM"],          # 
        ["HiWinter13", "AODSIM"],                 #
        ["pPb816Summer16DR", "AODSIM"],           #
        ["HINPbPbWinter16DR", "AODSIM"],          #
        ["HiFall13DR53X", "GEN-SIM-RECO"],        #    
        ["pAWinter13DR53X", "GEN-SIM-RECO"],      #
        ["XeXeRun2017", "*"],                     #
        ["HIRun2013A", "RECO"],                   #
        ["HINppWinter16DR", "AODSIM"],            #
        ["LowPU2010", "GEN-SIM"],                 #
        ["Run2010", "AOD"],                       # DPOA
        ["Run2010", "RAW-RECO"],                  # Keep cosmics
        ["Run2010", "RECO"],                      # Keep cosmics
        ["Run2011", "AOD"],                       #
        ["Run2011", "RAW-RECO"],                  #
        ["Run2011", "RECO"],                      #    
        ["Run2012", "AOD"],                       # DPOA
        ["Run2012", "RAW-RECO"],                  # Cosmics
        ["Summer11", "AODSIM"],                   #
        ["Summer11Leg", "GEN-SIM"],               #
        ["Fall11", "AODSIM"],                     #    
        ["Summer11LegDR", "AODSIM"],              #     
        ["Summer12DR53X", "AODSIM"],              #
        ["Summer11dr53X", "AODSIM"],              #
        ["Summer12_DR53X", "*"],                  #     
        ["Summer12", "AODSIM"],                   #
        ["Summer12", "GEN-SIM"],                  # 
        ["Summer13dr53X", "AODSIM"],              #
        ["Run2015C_25ns", "*"],                   #    
        ["Run2015", "AOD"],                       # DPOA
        ["Run2015", "RAW-RECO"],                  # Cosmics
        ["Phase2HLTTDRWinter20DIGI", "*"],        # Requested by TSG    
        ["Phase2HLTTDRSummer20ReRECOMiniAOD", "*"], #
        ["PhaseIITDRSpring17DR", "AOD"],          #
        ["PhaseIIFall16DR82", "AODSIM"],          #
        ["PhaseIISummer17wmLHEGENOnly", "LHE"],   # 
        ["GEM2019Upg14DR", "AODSIM"],             #
        ["PhaseISpring17DR", "AODSIM"],           # 
        ["PhaseIFall16DR", "AODSIM"],             #
        ["StreamExpressAlignment", "ALCARECO"],   #
        ["TkAlCosmicsInCollisions", "ALCARECO"],  #
        ["CMSSW_1_3_2", "RECO"],                  # What is this?
        ]

In [18]:
for ipair in campaign_tiers_to_keep :
    if ipair[1] not in global_exceptions[ipair[0]]:
        global_exceptions[ipair[0]].append(ipair[1])

In [19]:
datasets_strings = datasets.to_numpy(dtype="str")
tier_strings = tier.to_numpy(dtype="str")

### Go through all data tiers in the global exception list and keep those requested. 

In [20]:
for icampaign,itiers in global_exceptions.items():
    if "*" in itiers: 
        keep = np.where((np.core.defchararray.find(datasets_strings, icampaign) == -1),keep, True)
    else: 
        for itier in itiers: 
            found_campaign_in_dataset = (np.core.defchararray.find(datasets_strings, icampaign) != -1)
            tier_should_be_kept = np.where(tier == itier, True, False)
            keep = np.where(found_campaign_in_dataset & tier_should_be_kept,
                True, 
                keep)

# Current size breakdown

In [21]:
print('Total size               : %12.1f' % np.sum(size_on_tape))
print('Total size to keep       : %12.1f' % np.sum(size_on_tape[keep]))
print('Total size to delete     : %12.1f' % np.sum(size_on_tape[keep == False]))

Total size               :     226217.6
Total size to keep       :     144626.4
Total size to delete     :      81591.2


In [22]:
datasets_kept = datasets[keep]
datasets_categories_kept = np.full_like(datasets_kept, "")
size_on_tape_kept = size_on_tape[keep]

In [23]:
np.sum(size_on_tape_kept)

144626.38602

## Invalid exceptions

In [24]:
exception_valid = np.in1d(exception_list, datasets[keep == True])
exception_names = exception_list[exception_valid == False]

### Exceptions that don't seem to be on tape.

These are going to be safe and off of the "to delete" list, but just in case people are curious. 

In [25]:
for i in exception_names: 
    if "*" not in i : 
        print(i)

/JPsiToMuMu_5p02TeV_PbPb_NoHiMix_Incoherent_STARlight/HINPbPbAutumn18GS-103X_upgrade2018_realistic_HI_v11-v1/GEN-SIM
/JPsiToMuMu_OnOn_5p02TeV_PbPb_NoHiMix_Coherent_STARlight/HINPbPbAutumn18GS-103X_upgrade2018_realistic_HI_v11-v1/GEN-SIM
/HidjetQuenchedMinBias/Summer12-STARTHI44_V12-v5/GEN-SIM
/SingleMuon/Run2016D-SiPixelCalSingleMuon-21Feb2020_UL2016_HIPM_WMass-v2/ALCARECO
/SingleMuon/Run2016H-SiPixelCalSingleMuon-21Feb2020_UL2016_WMass-v2/ALCARECO
/SingleMuon/Run2016H-SiPixelCalSingleMuon-21Feb2020_UL2016_WMass-v3/ALCARECO
/StreamExpress/Run2018A-TkAlMinBias-Express-v1/ALCARECO
/StreamExpress/Run2018B-TkAlMinBias-Express-v1/ALCARECO
/StreamExpress/Run2018C-TkAlMinBias-Express-v1/ALCARECO
/StreamExpress/Run2018D-TkAlMinBias-Express-v1/ALCARECO
/Cosmics/Commissioning2021-TkAlCosmics0T-CRUZETmkFit-v1/ALCARECO
/Cosmics/Commissioning2022-TkAlCosmics0T-PromptReco-v1/ALCARECO
/AlCaLumiPixels/Run2018B-AlCaPCCZeroBias-PromptReco-v3/ALCARECO
/AlCaLumiPixels/Run2018E-AlCaPCCZeroBias-PromptReco-v

# Make formatted lists to keep and drop

In [26]:
df_keep = df[keep == True]
df_drop = df[keep == False]

In [27]:
## Stupid delimeter. 
delim = ''.join( ['-' for i in range(75)])

# Keep list

In [28]:
print( "%30s : %12s | %12s | %12s" % ( "Category", "T0", "T1", "Total" ) )
print(delim)
for category in categories:
    isum_t0 = np.sum( df_keep[ (df_keep["category"] == category) & (df_keep["rse_id"] == "T0_CH_CERN_Tape")]["size_on_tape_[TB]"] )
    isum_t1 = np.sum( df_keep[ (df_keep["category"] == category) & (df_keep["rse_id"] != "T0_CH_CERN_Tape")]["size_on_tape_[TB]"] )
    print( "%30s : %12.1f | %12.1f | %12.1f" % ( category, isum_t0, isum_t1, isum_t0 + isum_t1 ) )
print(delim)
sum_t0 = np.sum( df_keep[(df_keep["rse_id"] == "T0_CH_CERN_Tape")]["size_on_tape_[TB]"] )
sum_t1 = np.sum( df_keep[(df_keep["rse_id"] != "T0_CH_CERN_Tape")]["size_on_tape_[TB]"] )
print( "%30s : %12.1f | %12.1f | %12.1f" % ( "Total", sum_t0, sum_t1, sum_t0 + sum_t1 ) )

                      Category :           T0 |           T1 |        Total
---------------------------------------------------------------------------
                         Skims :        371.3 |       3692.0 |       4063.3
       Commissioning + Cosmics :         17.8 |        616.5 |        634.3
                         Run 3 :       3087.2 |       5475.7 |       8562.9
               Run 2 legacy MC :       8429.5 |      16774.4 |      25203.9
             Run 2 legacy data :       1362.2 |      14624.3 |      15986.4
         Run 2 pre-legacy data :       1515.5 |       6982.5 |       8498.0
           Run 2 pre-legacy MC :      12628.8 |      33768.6 |      46397.5
                     B-Parking :       2871.9 |        893.3 |       3765.2
                           HIN :       1059.1 |       9210.4 |      10269.5
                          DPOA :        708.6 |      10994.9 |      11703.4
                      Upgrades :       1021.3 |       6983.5 |       8004.8
            

In [29]:
keeping_sorted = df_keep.sort_values(by="size_on_tape_[TB]", ascending=False)

In [30]:
keeping_sorted.get("dataset", "size_on_tape_[TB]")

0         /Neutrino_E-10_gun/RunIISummer17PrePremix-PUFu...
125145    /Neutrino_E-10_gun/RunIISummer17PrePremix-PUAu...
109480    /Neutrino_E-10_gun/RunIISummer20ULPrePremix-UL...
35912     /Neutrino_E-10_gun/RunIISummer17PrePremix-MCv2...
109481    /Neutrino_E-10_gun/RunIISummer17PrePremix-MCv2...
                                ...                        
76388     /T_t-channel_TuneZ2star_8TeV-powheg-tauola/Sum...
124649    /HSCPmchamp3_M-900_TuneZ2star_8TeV-pythia6/Sum...
124671    /HTo2LongLivedTo2B2Mu_TripleMuEnrichedPt3_MH-1...
124674    /TTbarDMJets_DiLept_pseudoscalar_NLO_Mchi-1_Mp...
180597    /HTMHT/Run2015A-LogErrorMonitor-27Jan2016-v1/USER
Name: dataset, Length: 118172, dtype: object

In [31]:
df_keep.to_csv("keeping_sorted.csv")

# Drop list

In [32]:
print( "%30s : %12s | %12s | %12s" % ( "Category", "T0", "T1", "Total" ) )
print(delim)
for category in categories:
    isum_t0 = np.sum( df_drop[ (df_drop["category"] == category) & (df_drop["rse_id"] == "T0_CH_CERN_Tape")]["size_on_tape_[TB]"] )
    isum_t1 = np.sum( df_drop[ (df_drop["category"] == category) & (df_drop["rse_id"] != "T0_CH_CERN_Tape")]["size_on_tape_[TB]"] )
    print( "%30s : %12.1f | %12.1f | %12.1f" % ( category, isum_t0, isum_t1, isum_t0 + isum_t1 ) )
print(delim)
sum_t0 = np.sum( df_drop[(df_drop["rse_id"] == "T0_CH_CERN_Tape")]["size_on_tape_[TB]"] )
sum_t1 = np.sum( df_drop[(df_drop["rse_id"] != "T0_CH_CERN_Tape")]["size_on_tape_[TB]"] )
print( "%30s : %12.1f | %12.1f | %12.1f" % ( "Total", sum_t0, sum_t1, sum_t0 + sum_t1 ) )    

                      Category :           T0 |           T1 |        Total
---------------------------------------------------------------------------
                         Skims :          0.0 |          0.0 |          0.0
       Commissioning + Cosmics :         20.5 |        821.3 |        841.8
                         Run 3 :          0.0 |          0.0 |          0.0
               Run 2 legacy MC :       1427.0 |      19796.1 |      21223.1
             Run 2 legacy data :          0.0 |          0.0 |          0.0
         Run 2 pre-legacy data :        438.0 |       4069.5 |       4507.5
           Run 2 pre-legacy MC :        676.1 |       3253.1 |       3929.2
                     B-Parking :       1987.0 |        617.1 |       2604.1
                           HIN :         37.4 |        694.7 |        732.1
                          DPOA :        146.8 |       1036.2 |       1182.9
                      Upgrades :        647.6 |       4008.8 |       4656.4
            

In [33]:
dropping_sorted = df_drop.sort_values(by="size_on_tape_[TB]", ascending=False)
dropping_sorted_values = dropping_sorted["dataset"]

In [34]:
df_drop.to_csv("dropping_sorted.csv")

In [35]:
dropping_sorted

,Unnamed: 0,dataset,rse_id,DTier,size_on_tape_[TB],category
52981,52981,/Neutrino_E-10_gun/RunIISummer19ULPrePremix-UL...,T1_ES_PIC_Tape,PREMIX,697.62514,Run 2 legacy MC
76392,76392,/Neutrino_E-10_gun/RunIISummer19ULPrePremix-UL...,T1_IT_CNAF_Tape,PREMIX,656.97625,Run 2 legacy MC
125146,125146,/Neutrino_E-10_gun/RunIISummer19ULPrePremix-UL...,T1_US_FNAL_Tape,PREMIX,656.97625,Run 2 legacy MC
35913,35913,/Neutrino_E-10_gun/RunIISummer19ULPrePremix-UL...,T1_DE_KIT_Tape,PREMIX,635.58527,Run 2 legacy MC
125148,125148,/Neutrino_E-10_gun/RunIISummer19ULPrePremix-UL...,T1_US_FNAL_Tape,PREMIX,493.18779,Run 2 legacy MC
...,...,...,...,...,...,...
76238,76238,/QBH_ADD_MD_10_MQBH_12_n_4_13TeV_TuneCUETP8M1-...,T1_FR_CCIN2P3_Tape,LHE,0.00000,Unknown
76237,76237,/BlackHole_BH9_MD8000_MBH11000_n6_13TeV_TuneCU...,T1_FR_CCIN2P3_Tape,LHE,0.00000,Unknown
76236,76236,/GluGluToHToWWTo2Tau2Nu_M-350_7TeV-powheg-pyth...,T1_FR_CCIN2P3_Tape,DQM,0.00000,DPOA
76235,76235,/BlackHole_BH2_MD3000_MBH7000_n2_13TeV_TuneCUE...,T1_FR_CCIN2P3_Tape,LHE,0.00000,Unknown


# Total to remove from T0

In [36]:
np.sum( dropping_sorted["size_on_tape_[TB]"][dropping_sorted["rse_id"] == "T0_CH_CERN_Tape"] )

12414.16175

# Total to remove from T1

In [37]:
np.sum( dropping_sorted["size_on_tape_[TB]"] ) - np.sum( dropping_sorted["size_on_tape_[TB]"][dropping_sorted["rse_id"] == "T0_CH_CERN_Tape"] )

69177.00991000001

## DPOA Data

In [38]:
vals_dpoa = keeping_sorted["size_on_tape_[TB]"][keeping_sorted["category"] == "DPOA"][keeping_sorted["rse_id"] == "T0_CH_CERN_Tape"]

In [39]:
np.sum( vals_dpoa )

708.55211

## Sanity check of exceptions

List of exceptions should not be in the list to drop. The intersection of these sets should be empty. 

In [40]:
set(exception_list) & set(dropping_sorted_values)

set()